In [19]:
import sys
sys.path.append('../../')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import utilities

# begin by opening and examining the SP500 companies


tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
print(tickers["Symbol"].values)

['MMM' 'AOS' 'ABT' 'ABBV' 'ACN' 'ADBE' 'AMD' 'AES' 'AFL' 'A' 'APD' 'ABNB'
 'AKAM' 'ALB' 'ARE' 'ALGN' 'ALLE' 'LNT' 'ALL' 'GOOGL' 'GOOG' 'MO' 'AMZN'
 'AMCR' 'AEE' 'AAL' 'AEP' 'AXP' 'AIG' 'AMT' 'AWK' 'AMP' 'AME' 'AMGN' 'APH'
 'ADI' 'ANSS' 'AON' 'APA' 'AAPL' 'AMAT' 'APTV' 'ACGL' 'ADM' 'ANET' 'AJG'
 'AIZ' 'T' 'ATO' 'ADSK' 'ADP' 'AZO' 'AVB' 'AVY' 'AXON' 'BKR' 'BALL' 'BAC'
 'BK' 'BBWI' 'BAX' 'BDX' 'BRK.B' 'BBY' 'BIO' 'TECH' 'BIIB' 'BLK' 'BX' 'BA'
 'BKNG' 'BWA' 'BXP' 'BSX' 'BMY' 'AVGO' 'BR' 'BRO' 'BF.B' 'BLDR' 'BG'
 'CDNS' 'CZR' 'CPT' 'CPB' 'COF' 'CAH' 'KMX' 'CCL' 'CARR' 'CTLT' 'CAT'
 'CBOE' 'CBRE' 'CDW' 'CE' 'COR' 'CNC' 'CNP' 'CDAY' 'CF' 'CHRW' 'CRL'
 'SCHW' 'CHTR' 'CVX' 'CMG' 'CB' 'CHD' 'CI' 'CINF' 'CTAS' 'CSCO' 'C' 'CFG'
 'CLX' 'CME' 'CMS' 'KO' 'CTSH' 'CL' 'CMCSA' 'CMA' 'CAG' 'COP' 'ED' 'STZ'
 'CEG' 'COO' 'CPRT' 'GLW' 'CTVA' 'CSGP' 'COST' 'CTRA' 'CCI' 'CSX' 'CMI'
 'CVS' 'DHR' 'DRI' 'DVA' 'DE' 'DAL' 'XRAY' 'DVN' 'DXCM' 'FANG' 'DLR' 'DFS'
 'DG' 'DLTR' 'D' 'DPZ' 'DOV' 'DOW' 'DHI' 'DTE' 'DUK' '

In [20]:
def retrieve_data(ticker):
    filename = "../../data/stocks/{}_full_1min_adjsplit.txt".format(ticker)

    try:
        df = pd.read_csv(filename, sep=",", header=0)
    except FileNotFoundError:
        print("{}: file not found".format(filename))
        return None

    df.columns = ["timestamp", "open", "high", "low", "close", "volume"]
    df["ticker"] = ticker
    df = utilities.timestamp_to_date_and_time(df)
    
    misses = utilities.missing_markethours(df)
    
    print(ticker, misses)
    return misses

In [21]:
import os

# replace tickers with the whole list
files = os.listdir('../../data/stocks')
tickers = [f.split('_')[0] for f in files if f.endswith('.txt')]
print(len(tickers))

6567


In [22]:
all_misses = pd.DataFrame(columns=["ticker", "miss_rate"])

for ticker in tickers:
    misses = retrieve_data(ticker)
    new_row = pd.DataFrame({"ticker": ticker, "miss_rate": [misses]})
    all_misses = pd.concat([all_misses, new_row])

ILMN 0.07986546342453826
SQ 0.006689829516444368
DG 0.02792094555318736
RANI 0.8260949576739051
NMTC 0.8291285103785104
DNA 0.03253365613880632
PMN 0.9462647262647262


KeyboardInterrupt: 

In [ ]:
all_misses.to_csv("all_ticker_misses.csv", index=False)